# 使用Python truffle 构建完整的智能合约
### 参见：https://www.aifirm.org/2019-07-31-Use-Python-Truffle-Building-Smart-Contract/

In [1]:
import json
import copy
from web3 import Web3,HTTPProvider
from solc import compile_source

In [2]:
web3 = Web3(Web3.HTTPProvider('http://192.168.31.50:9545'))

In [3]:
accounts = web3.eth.accounts
accounts

['0xDe560d0FAC79E7d35e58BBf3BB2f138586975Ecc',
 '0xBEC2A0dBbA58C3B52CBb12690113eA43CE42E27c',
 '0xb20D15F5D7AD18edBa27018bA9BF6C0c9Cb25CCD',
 '0x0AE90087a3A8b85e25E9f7eBf10E260A2EB57d2D',
 '0xA868386f043517ba163Ae9fC16FC1ceE5d0fAe54',
 '0xdb1d4624DCF81F67a529Ac72C440A8b10ad0E378',
 '0xdEDc486F54ce9d2A27918cd201e3A7B725244797',
 '0x0CF06A81204B04C451ECD913a3c95A723FB19AE7',
 '0x8a42e80A595Cd6008441ced671f4cb7537b69De2',
 '0xa4B2D413BbC1ec191db9bDAc46cE4E886507F494']

## 利用truffle compile 生成 abi ，truffle migrate 部署合约，得到合约地址
#### 打开build/contracts/ModelRepo.json
1. turffle compile
2. 在 ./build/contracts/ModelRepo.json 文件中找到 abi部分，放到http://www.bejson.com/jsoneditoronline/ 中格式化一下， 去掉 “abi“， 和最后面的 ","
3. 或者使用 https://remix.ethereum.org/#optimize=false&appVersion=0.7.7&version=soljson-v0.5.1+commit.c8a2cb62.js 在线 compile 生成的 api 
4. 将2或3步骤中的abi内容放到 ./abis/ModelRepo.abi 中
5. truffle migrate 部署合约
6. 在 ./build/contracts/ModelRepo.json 文件中找到 network 部分的 address， 将合约地址 address 设置到下面的 contract_address 变量中

In [ ]:
class ModelRepository():

    def __init__(self, contract_address, account=None):
 
       
        self.web3 = Web3(Web3.HTTPProvider('http://192.168.31.245:9545'))

        if account is not None:
            self.account = account
        else:
            print("No account submitted... using default[2]")
            self.account = self.web3.eth.accounts[2]

        self.connect_to_contract(contract_address)

    def connect_to_contract(self, contract_address):
        """Connects to the Sonar contract using its address and ABI"""

        f = open('./abis/ModelRepo.abi', 'r')
        abi = json.loads(f.read())
        f.close()

        self.contract = self.web3.eth.contract(abi=abi, address=contract_address)

        self.contract_address = contract_address

        print("Connected to OpenMined ModelRepository:" +
              str(self.contract_address))


#### 打开build/contracts/ModelRepo.json
 找到 "networks": {
    "5777": {
      "events": {},
      "links": {},
      "address": "0x323E652A65aA1a96E270b9f8B3e3BB2Edb49EBf3",
      "transactionHash": "xxx"
    }
 "address": "0x7034cE25eC9401B6F676ef296B7231a3840F65b5" 就是合约地址：  0x323E652A65aA1a96E270b9f8B3e3BB2Edb49EBf3
 
 0x323E652A65aA1a96E270b9f8B3e3BB2Edb49EBf3

In [35]:
contract_address = "0x323E652A65aA1a96E270b9f8B3e3BB2Edb49EBf3"

In [ ]:
repo = ModelRepository(contract_address = contract_address)

In [ ]:
gas_estimate = repo.contract.functions.setInitErr(383).estimateGas()
gas_estimate

### set 方法

In [ ]:
txn = {}
txn["from"] = repo.account
txn["to"] = repo.contract_address

repo.contract.functions.setInitErr(444).transact(txn)

### get 方法

In [ ]:
repo.contract.caller.getModelInitErr()

In [ ]:
repo.contract.functions.setModel(111,222).transact(txn)

In [ ]:
repo.contract.caller.getModel()

## 直接利用web3.py solc 生成 abi ，部署使用 truffle migrate，然后拷贝合约地址

#### 1. 代码参看 web3.py contract ： https://web3py.readthedocs.io/en/stable/examples.html#working-with-contracts
#### 2. 将合约文件 ModelRepo.sol 拷贝到 ./abis/ 中

In [41]:
from solc import compile_source

In [44]:
def compile_source_file(file_path):
    with open(file_path, 'r') as f:
        source = f.read()

    return compile_source(source)


In [45]:
conpile_source = compile_source_file('./abis/ModelRepo.sol')

In [46]:
contract_id, contract_interface = conpile_source.popitem()

In [55]:
address = "0x323E652A65aA1a96E270b9f8B3e3BB2Edb49EBf3"  # 合约地址

In [48]:
modelrepo_contract = web3.eth.contract(
   address = address,
   abi=contract_interface['abi'])

In [51]:
modelrepo_contract.caller.getModelInitErr()

555

In [53]:
txn = {}
txn["from"] = web3.eth.accounts[2]
txn["to"] = address
modelrepo_contract.functions.setInitErr(666).transact(txn)

HexBytes('0xe90c33f81a90403e9e923954c17a56f85e24ab09dc27f4e931b6eb3d7af77568')

In [54]:
modelrepo_contract.caller.getModelInitErr()

666